In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/Colab\ Notebooks/FSI/Fruits/

/content/drive/My Drive/Colab Notebooks/FSI/Fruits


# **Base Fruits 360**

A base fruits 360 é formada por imagens de frutas e vegetais, contendo 120 classes diferentes.

![alt text](https://predictiveprogrammer.com/wp-content/uploads/2018/06/dataset.png)

Para cada classe existem por volta de 500 imagens de treinamento. Cada imagem tem dimensão 100x100 pixels e é colorida. Nas imagens as frutas estão posicionadas de forma diferente e em diversas rotações.

Vemos então que existe um problema mais complexo que o da base Mnist e não faz muito sentido tratar a imagem em si como entrada do classificador.



## **Abordagem de Solução**

Como as imagens não devem ser usadas diretamente como atributos dos classificadores, decidimos fazer extração de features da imagem. 

Mas o que são features?

Features podem ser interpretadas como padrões na imagens: cantos, listras, texturas

Como extraimos essas features?

Existem várias formas de fazer detecção de features, nós usaremos um detector chamado **orb**





### **Orb Detector (Oriented FAST and Rotated BRIEF)**

O descritor orb utilizado é implementado na biblioteca de visão computacional OpenCV. 

Ele detecta pontos de interesse na imagem (keypoints) e para cada ponto gera um descritor de 32 bytes.

Passaremos as imagens para o extrator de features, cada imagem irá retornar por volta de 10 descritores. Os descritores serão usados como entrada de atributos dos classificadores.

Na fase de predição e teste, faremos novamente a extração de features, para cada feature o classificador fará uma predição, a classificação da imagem será a moda da classificação dos seus descritores.

## **Implementação**

Imports necessários para o código.



In [0]:
import sys 
import cv2
import numpy as np
import os
import glob
import pandas as pd
import random
import pickle
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm
from sklearn.linear_model import Perceptron
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix
from scipy import stats

Importa arquivo de configuração com as informações da biblioteca, caminho das pastas e nome das classes que serão estudadas.

Como a base é muito extensa, fizemos arquivos que importam o problema para 5, 10, 40 e 120 classes. 

In [0]:
sys.path.append('.')
import data_conf_5 as dc

Função que plota a matriz de confusão.

In [0]:
def plot_confusion_matrix(y_true, y_pred, classes, title):
    acc = accuracy_score(y_true, y_pred)
    title = title + " (Acurácia: " + str("{:10.4f}".format(acc)) + ")"

    cm = confusion_matrix(y_true, y_pred, classes, normalize='true')
    cm_df = pd.DataFrame(cm, index = classes, columns = classes)
    plt.figure(figsize=(5.5,4))
    sns.heatmap(cm_df, annot=True, cmap="YlGnBu")
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

Função que carrega as imagens das pastas listadas no arquivo de configuração.

In [0]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    
    images = np.array(images)
    return images

Função que extrai as features das imagens.

In [0]:
def extract_features(image): 
    # Transforma imagem em grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    # Instancia um descritor orb
    orb = cv2.ORB_create(scoreType=cv2.ORB_HARRIS_SCORE,edgeThreshold=10,nfeatures=10)
    kp = orb.detect(gray, None)
    kp, des = orb.compute(gray, kp) 
    des = np.array(des)

    return des

Função que realiza o treinamento dos classificadores.

In [0]:
def training(clf, filename):
    print("Extraindo features das imagens ...")

    # Varre as pastas selecionadas no arquivo data_conf.py
    for i in range(len(dc.folder_path_train)):
        # Carrega imagens da pasta
        images = load_images_from_folder(dc.folder_path_train[i])

        # Para cada imagem, retira os descritores orb da imagem e coloca na lista de descritores
        for j in range(len(images)):
            des = extract_features(images[j])
            if(des.shape!=()):
                if(j == 0):
                    des_list = des
                else:
                    des_list = np.vstack((des_list, des))

        # Cria a matriz de atributos e o vetor de labels
        if(i == 0):
            atributes = des_list
            labels = np.full((1,des_list.shape[0]),dc.class_names[i])
        else:
            atributes = np.vstack((atributes, des_list))
            labels = np.append(labels, np.full((1,des_list.shape[0]),dc.class_names[i]))

    # Treina classificador
    print("Inicio do treinamento ...")
    t0 = time.time()
    clf.fit(atributes, labels)
    t1 = time.time()
    print("O treinamento demorou", t1-t0, " segundos")
    pickle.dump(clf, open((filename+".sav"), 'wb'))

Função que gera as predições do modelo.

In [0]:
def test(filename):

    # Carrega o classificador
    clf = pickle.load(open((filename+".sav"), 'rb'))

    t0 = time.time()
    for i in range(len(dc.folder_path_test)):
        images = load_images_from_folder(dc.folder_path_test[i])

        for j in range(len(images)):
            # Extrai as features da imagem
            des = extract_features(images[j])

            # Se existirem features, faz a predição do classificador das features, se não excolhe uma classe aleatoriamente
            if(des.shape != ()):
                prediction = clf.predict(des) # Vetor de classificação das features
                # A classe da imagem será a moda das classificações das features
                prediction = stats.mode(prediction).mode[0]
            else:
                prediction = np.random.choice(dc.class_names)

            if(i==0 and j==0):
                y_pred = prediction
                y_true = dc.class_names[i]
            else:
                y_pred = np.append(y_pred,prediction)
                y_true = np.append(y_true,dc.class_names[i])

    t1 = time.time()
    print("A fase de predição e teste durou", t1-t0, " segundos")
    plot_confusion_matrix(y_true, y_pred, classes=dc.class_names,title='Confusion matrix ')

Função menu que inicializa os classificadores.

In [0]:
def menu(mode, mult_clf_mode, bin_clf_mode):

    # Define o classificador binário
    if(bin_clf_mode == "--svc"):
        bin_clf = svm.SVC(class_weight = 'balanced')
        filename = "svc"
    elif(bin_clf_mode == "--mlp"):
        bin_clf = MLPClassifier()
        filename = "mlp"
    else:
        print("Escolha o terceiro argumento como --svc ou --mlp")
        exit()

    # Define o classificador multiclasse
    if(mult_clf_mode == "--ovr"):
        mult_clf = OneVsRestClassifier(bin_clf, n_jobs = -1)
        filename = "ovr_"+filename
    elif(mult_clf_mode == "--ovo"):
        mult_clf = OneVsOneClassifier(bin_clf, n_jobs = -1)
        filename = "ovo_"+filename
    elif(mult_clf_mode == "--eoc"):
        mult_clf = OutputCodeClassifier(bin_clf,code_size =3.0, n_jobs=-1)
        filename = "eoc_"+filename
    else:
        print("Escolha o segundo argumento como --ovr ou --ovo ou --eoc")
        exit()

    if(mode == "--train"):
        training(mult_clf,filename)
    elif(mode == "--test"):
        test(filename)
    else:
        print("Escolha o primeiro argumento como --train ou --test")
        exit()

## Resultados

Para fazer a demonstração do código, utilizaremos apenas 5 classes de frutas

In [0]:
menu("--train", "--ovr", "--svc")
menu("--test", "--ovr", "--svc")

### One-vs-Rest

Fizemos o teste também escolhendo 10 classes de frutas.

Tempo de treinamento: 7min e 42s

Tempo de predição: 1min e 27s

![alt text](https://drive.google.com/uc?id=1cqCR6b71FaElfNNyk0iHC6SBCa6YImPT)

## One-vs-One

Fizemos o teste escolhendo 10 classes de frutas.

Tempo de treinamento: 45.6s

Tempo de predição: 4min e 7s


![alt text](https://drive.google.com/uc?id=1XcS6HgSUwE_KEYVFMqx2nhESU3dcSP3o)

Fizemos o teste também escolhendo 40 classes de frutas.

Tempo de treinamento: 16min

Tempo de predição: 7h

![alt text](https://drive.google.com/uc?id=14ap57W1VSfNEdFPLgagGcplXJo6L1k6C)


## Error Output Code

Fizemos o teste também escolhendo 10 classes de frutas.

Tempo de treinamento: 36min

Tempo de predição: 11min

![alt text](https://drive.google.com/uc?id=1J1MDasAeFeR7UbRCYb1rhymwDcSy8Kem)